In [1]:
import sqlite3

# เชื่อมต่อกับไฟล์ฐานข้อมูล
conn = sqlite3.connect('password_data.sqlite')
cursor = conn.cursor()

In [2]:
import pandas as pd

# อ่านข้อมูลจากตาราง
query = "SELECT * FROM users;"
df = pd.read_sql_query(query, conn)

# แสดงข้อมูล
print(df.head(10))

   index                 password  strength
0      0                zxe870819         1
1      1             xw46454nr23l         1
2      2                soporte13         1
3      3  accounts6000webhost.com         2
4      4                 c443balg         1
5      5                16623670p         1
6      6                 yj9q3f8p         1
7      7          180ZIRUVIcuFERy         2
8      8                 djredd09         1
9      9                yin172015         1


In [3]:
# ลบคอลัมน์ 'index'
df = df.drop('index', axis=1)

# แสดงผลลัพธ์หลังจากลบคอลัมน์
print(df.head(10))

                  password  strength
0                zxe870819         1
1             xw46454nr23l         1
2                soporte13         1
3  accounts6000webhost.com         2
4                 c443balg         1
5                16623670p         1
6                 yj9q3f8p         1
7          180ZIRUVIcuFERy         2
8                 djredd09         1
9                yin172015         1


In [4]:
import joblib

# โหลดโมเดล
model = joblib.load('lr_model.pkl')

# ตรวจสอบว่าโมเดลโหลดสำเร็จ
print(type(model))

<class 'sklearn.linear_model._logistic.LogisticRegression'>


In [6]:
import string

feature_order = ['length', 'uppercase', 'lowercase', 'digits', 'special_chars']

def extract_features(password):
    """แปลงรหัสผ่านให้เป็นฟีเจอร์"""
    features = {
        'length': len(password),
        'digits': sum(c.isdigit() for c in password),
        'uppercase': sum(c.isupper() for c in password),
        'lowercase': sum(c.islower() for c in password),
        'special_chars': sum(c in string.punctuation for c in password),
    }
    return pd.Series(features)

# แปลงข้อมูลทั้งหมดในคอลัมน์ 'password'
feature_df = df['password'].apply(extract_features)
feature_df = feature_df[feature_order]  # จัดเรียงคอลัมน์ตามลำดับที่โมเดลต้องการ

In [8]:
from sklearn.metrics import accuracy_score
# ค่าจริงจากข้อมูล
y_true = df['strength']  # สมมติว่าคอลัมน์ 'label' คือค่าจริง

# ค่าที่โมเดลพยากรณ์
y_pred = model.predict(feature_df)

# คำนวณค่า accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy of the model on unseen data: {accuracy:.2f}")

Accuracy of the model on unseen data: 1.00
